In [8]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')
import viz
import pickle as pkl
import matplotlib.pyplot as plt
from os.path import join as oj
import os
import numpy as np
import imodels
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier

import data
import validate
outcome_def = 'iai_intervention' # output
MODELS_DIR = '../models/simple_3_splits'
os.makedirs(MODELS_DIR, exist_ok=True)


# load the data
df_pecarn, df_psrc, common_feats, filtered_feats_pecarn, filtered_feats_psrc = \
    data.load_it_all(dummy=True, frac_missing_allowed=0.05)
common_feats += ['iai', 'CTScan']
df = df_pecarn[common_feats + ['Sex_M']].append(df_psrc[common_feats])
processed_feats = data.select_final_feats(common_feats)

# split the idxs
train_idxs = df.cv_fold.isin(data.pecarn_train_idxs)
test_idxs1 = df.cv_fold.isin(data.pecarn_test_idxs)
test_idxs2 = df.cv_fold.isin(data.psrc_train_idxs + data.psrc_test_idxs)

# split the data
X, y = df[processed_feats], df[outcome_def]
half = train_idxs.sum() // 2
split_to_plot = '_test2'

# 3-split
pec = df[train_idxs | test_idxs1]
train = df[train_idxs]
test1 = df[test_idxs1]
test2 = df[test_idxs2]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/accounts/projects/vision/.local/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/accounts/projects/vision/.local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
dsets = [pec, train, test1, test2]

In [10]:
list(df_pecarn.keys())

['id',
 'AbdCTScan',
 'AbdCTScan1',
 'AbdUltrasound',
 'AbdUltrasound1',
 'AbnChestAusc',
 'AlcDrgSuspicion',
 'BowelSounds',
 'Certification',
 'ChestCXR',
 'CurrIntubated',
 'EDTriageTime',
 'FlankTender',
 'GCSEye',
 'GCSMotor',
 'GCSScore',
 'GCSVerbal',
 'HRDocumented',
 'IAIAcuteInterv',
 'IAISuspicion',
 'InitHeartRate',
 'InitResp',
 'InitRespRange',
 'InitSysBP',
 'InitSysBPRange',
 'InjTmKnown',
 'InjuryDay',
 'InjuryMechanism',
 'LabResultAware',
 'LabsObtained',
 'LtCostalTender',
 'NonAbdSurgery',
 'OthAbdImages',
 'PelvicTender',
 'PelvicXR',
 'PelvisUnstable',
 'PeritonIrrit',
 'PharmaParalyzed',
 'PharmaSedated',
 'Preverbal',
 'PtHosp',
 'RectalBlood',
 'RefuseFU',
 'RtCostalTender',
 'ShortBreath',
 'TriageTmUnkown',
 'Age',
 'iai',
 'iai_intervention',
 'CTScan',
 'cv_fold',
 'AbdDistention_no',
 'AbdDistention_unknown',
 'AbdDistention_yes',
 'AbdTenderDegree_Mild',
 'AbdTenderDegree_Moderate',
 'AbdTenderDegree_None',
 'AbdTenderDegree_Severe',
 'AbdTenderDegree_un

In [17]:
feats = ['Age<2_yes', 'iai_intervention', 'iai', 
         'MOI_Motor vehicle collision', 'MOI_Fall from an elevation', 'MOI_Bike collision/fall',
         'CTScan']
for dd, d in zip(['pec', 'train', 'test1', 'psrc'], dsets):
    print(dd)
    print('\t', 'num', d.shape[0])
    try:
        k = 'Sex_M'
        print(f'\t {k} {int(d[k].sum())} ({100*d[k].sum()/d.shape[0]:.1f}%)')
    except:
        pass
    for k in feats:
        print(f'\t {k} {int(d[k].sum())} ({100*d[k].sum()/d.shape[0]:.1f}%)')

pec
	 num 12044
	 Sex_M 7384 (61.3%)
	 Age<2_yes 1167 (9.7%)
	 iai_intervention 203 (1.7%)
	 iai 761 (6.3%)
	 MOI_Motor vehicle collision 3832 (31.8%)
	 MOI_Fall from an elevation 2161 (17.9%)
	 MOI_Bike collision/fall 767 (6.4%)
	 CTScan 5179 (43.0%)
train
	 num 7985
	 Sex_M 4887 (61.2%)
	 Age<2_yes 761 (9.5%)
	 iai_intervention 133 (1.7%)
	 iai 485 (6.1%)
	 MOI_Motor vehicle collision 2505 (31.4%)
	 MOI_Fall from an elevation 1471 (18.4%)
	 MOI_Bike collision/fall 489 (6.1%)
	 CTScan 3393 (42.5%)
test1
	 num 4059
	 Sex_M 2497 (61.5%)
	 Age<2_yes 406 (10.0%)
	 iai_intervention 70 (1.7%)
	 iai 276 (6.8%)
	 MOI_Motor vehicle collision 1327 (32.7%)
	 MOI_Fall from an elevation 690 (17.0%)
	 MOI_Bike collision/fall 278 (6.8%)
	 CTScan 1786 (44.0%)
psrc
	 num 2188
	 Sex_M 0 (0.0%)
	 Age<2_yes 216 (9.9%)
	 iai_intervention 62 (2.8%)
	 iai 261 (11.9%)
	 MOI_Motor vehicle collision 1014 (46.3%)
	 MOI_Fall from an elevation 155 (7.1%)
	 MOI_Bike collision/fall 77 (3.5%)
	 CTScan 967 (44.2%)


In [16]:
test2['Age'].describe()

count    2188.000000
mean        7.797265
std         4.634462
min         0.000000
25%         4.000000
50%         8.000000
75%        12.000000
max        15.000000
Name: Age, dtype: float64